In [ ]:
#Importar librerías
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64


# Cargue y procesamiento de datos de datos
df = pd.read_csv('BASE_FINAL_V2.csv', delimiter = ';')
group_gravedad = df.groupby(['Fecha', 'GravedadNombre'], as_index = False)['CountGravedadNombre'].count()
group_CausasAcc = df.groupby(['CausasAcc','GravedadNombre'], as_index = False)['CountCausasAcc'].count()
all_gravedad = df.GravedadNombre.unique()
all_gravedad1 = group_CausasAcc.GravedadNombre.unique()
group_HoraProcesada = df.groupby(['HoraProcesada','GravedadNombre'], as_index = False)['CountHoraProcesada'].count()
all_HoraProcesada = group_HoraProcesada.GravedadNombre.unique()
group_Fecha = df.groupby(['Fecha', 'MesProcesado'], as_index = False)['CountGravedadNombre'].count()
all_MesProcesado = df.MesProcesado.unique()

# dash bootstrap components
# https://dash-bootstrap-components.opensource.faculty.ai/docs/
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# Imagenes
image_filename = 'logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


# los argumentos de estilo para la barra lateral (Posición: fija y un ancho fijo.)
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# los estilos del contenido principal a la derecha de la barra lateral
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

# Barra lateral
sidebar = html.Div(
    [
        html.H4("Siniestros Bogotá", className="display-4", style = {'textAlign' : 'center'}),
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'18%'}),
        html.Hr(),
        html.P(
            "Haga clic para visualizar el comportamiento de los siniestros viales", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("Introducción", href="/", active="exact"),
                dbc.NavLink("Dahsboards 1", href="/page-1", active="exact"),
                dbc.NavLink("Dahsboards 2", href="/page-2", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

# Contenido del Dahs
content = html.Div(id="page-content", style=CONTENT_STYLE)

# Se invocan elementos sidebar y content
app.layout = html.Div([dcc.Location(id="url"), sidebar, content])

# Ajustar bordes fijos https://stackoverflow.com/questions/60170645/subplot-side-to-side-in-python-dash
@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return html.Div([
                        html.H1('Introducción', style = {'textAlign' : 'center'}),
                        html.Hr(),
                        html.Div([
                        html.P('Bogotá es la ciudad con mayor población del país, 8.080.734 habitantes que corresponden al 19 % de la población totalde Colombia.'),
                        html.P('Las tasas de muerte por accidentes de transporte, que incluyen todos los eventos de transporte terrestre, fluvial, marítimo,aéreo y férreo, corresponden al 26,6 % de las muertes violentas en el ámbito nacional en 2017. Los accidentes detransporte son la segunda causa más importante de muerte violenta en el país, después del homicidio . En la ciudadde Bogotá, el 21.7 % de las muertes violentas fueron por accidentes de transporte, ubicándola como la quinta ciudadlatinoamericana con más muertos por accidentes de tránsito con más de 1.000.000 de habitantes, solo por detrás de Cali,Medellín, Cartagena y Barranquilla.'),
                        html.P('Desde el año 2003, Bogotá ha mantenido una tasa anual aproximada de 600 muertos por siniestros viales.Del total de fatalidades, el 71 % son peatones y motociclistas.'),
                        html.Hr(),
                        ]
                        ),
                        html.Div([
                            html.P('A continuación el comportamiento de los siniestros a analizar:'),
                        ]
                        ),
                        dbc.Row(
                                 [
                                    dbc.Col(html.Div([
                                                        dcc.Checklist(
                                                            id="checklist1",
                                                            options=[{"label": x, "value": x} 
                                                                     for x in all_MesProcesado],
                                                            value=all_MesProcesado[:],
                                                            labelStyle={'display': 'inline-block'}
                                                        ),
                                                        dcc.Graph(id="line-chart1"),
                                                    ])),
                                 ]
                            ),
                    ])
    elif pathname == "/page-1":
        return html.Div(
                        [
                          dbc.Row(
                                [
                                    dbc.Col(html.Div([
                                                    html.P("Seleccione Gravedad, Localidad, Día o Mes:"),
                                                    dcc.Dropdown(
                                                        id='names', 
                                                        value='GravedadNombre', 
                                                        options=[{'value': x, 'label': x} 
                                                                 for x in ['GravedadNombre','Localidad','Dia','MesProcesado']],
                                                        clearable=False
                                                    ),
                                                    #html.P("Values:"),
                                                    dcc.Dropdown(
                                                        id='values', 
                                                        value='Sector', 
                                                        options=[{'value': x, 'label': x} 
                                                                 for x in ['Sector']],
                                                        clearable=False
                                                    ),
                                                    dcc.Graph(id="pie-chart", style={"width":500, "margin": 0, 'display': 'inline-block'}),
                                                ])),
                                    dbc.Col(html.Div([
                                                    dcc.Checklist(
                                                        id="checklist",
                                                        options=[{"label": x, "value": x} 
                                                                 for x in all_gravedad],
                                                        value=all_gravedad[1:2],
                                                        labelStyle={'display': 'inline-block'}
                                                    ),
                                                    dcc.Graph(id="line-chart", style={"width":500, "margin": 0, 'display': 'inline-block'}),
                                                ])),
                                ]
                            ),
                            dbc.Row(
                                [
                                    dbc.Col(html.Div([
                                                    dcc.Dropdown(
                                                        id="dropdown1",
                                                        options=[{"label": x, "value": x} for x in all_HoraProcesada],
                                                        value=all_HoraProcesada[0],
                                                        clearable=False,
                                                    ),
                                                    dcc.Graph(id="bar-chart1"),
                                                ])),
            
                                ]
                            ),
                        ]
                    )
    elif pathname == "/page-2":
        return html.Div(
                        [
                          dbc.Row(
                                [
                                    dbc.Col(html.Div([
                                                    dcc.Dropdown(
                                                        id="dropdown",
                                                        options=[{"label": x, "value": x} for x in all_gravedad1],
                                                        value=all_gravedad1[1],
                                                        clearable=False,
                                                    ),
                                                    dcc.Graph(id="bar-chart"), 
                                                ])),
                                ]
                            ),
                            dbc.Row(
                                [
                                    #dbc.Col(html.Div("One of three columns")),
                                    #dbc.Col(html.Div("One of three columns")),
                                    #dbc.Col(html.Div(fig = px.sunburst(df, path=['GravedadNombre', 'TipoDisenno']))),
                                ]
                            ),
                        ]
                    ),
                        
# Si el usuario intenta llegar a una página diferente, devuelve un mensaje 404
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

# callback pie-chart Gráfico Introducción
@app.callback(
    Output("line-chart1", "figure"), 
    [Input("checklist1", "value")])
def update_line_chart(MesProcesados):
    mask = group_Fecha.MesProcesado.isin(MesProcesados)
    fig = px.line(group_Fecha[mask], 
        x="Fecha", y="CountGravedadNombre", color='MesProcesado')
    return fig

# callback pie-chart Gráfico 1
@app.callback(
    Output("pie-chart", "figure"), 
    [Input("names", "value"), 
     Input("values", "value")])
def generate_chart(names, values):
    fig = px.pie(df, values=values, names=names)
    return fig

# callback Line chart Gráfico 2
@app.callback(
    Output("line-chart", "figure"), 
    [Input("checklist", "value")])
def update_line_chart(GravedadNombres):
    mask = group_gravedad.GravedadNombre.isin(GravedadNombres)
    fig = px.line(group_gravedad[mask], 
        x="Fecha", y="CountGravedadNombre", color='GravedadNombre')
    return fig

# callback Bar chart Gráfico 3
@app.callback(
    Output("bar-chart1", "figure"), 
    [Input("dropdown1", "value")])
def update_bar_chart(GravedadNombre):
    mask = group_HoraProcesada["GravedadNombre"] == GravedadNombre
    fig = px.bar(group_HoraProcesada[mask], x="HoraProcesada", y="CountHoraProcesada", 
                 color="HoraProcesada", barmode="group")
    return fig

# callback Bar chart Gráfico 4
@app.callback(
    Output("bar-chart", "figure"), 
    [Input("dropdown", "value")])
def update_bar_chart(GravedadNombre):
    mask = group_CausasAcc["GravedadNombre"] == GravedadNombre
    fig = px.bar(group_CausasAcc[mask], x="CausasAcc", y="CountCausasAcc", 
                 color="CausasAcc", barmode="group")
    return fig


if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/May/2021 19:04:04] "GET /page-1 HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 19:04:05] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 19:04:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 19:04:05] "GET /_favicon.ico?v=1.20.0 HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 19:04:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 19:04:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 19:04:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 19:04:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 19:05:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 19:05:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 19:05:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 19:05:33] "POST /_dash-update-compo